### Read a month of .HXV files

In [ ]:
using CSV
using NativeFileDialog
using Glob
using Dates, DataFrames, Distributions, DSP
using LaTeXStrings
using Printf
using Statistics #, StatsPlotss
using StatsBase

##include("../Split_Spectra/Split_Spectra_Tools.jl")
include("C://Users//PC1//Julia_programs//Split_Spectra//Split_Spectra_Tools.jl")
# Widen screen for better viewing
display(HTML("<style>:root { --jp-notebook-max-width: 80% !important; }</style>"))

const sample_frequency = 1.28

# temporary code to handle spike in data for the Townsville record 25/01/2004 22:30              
function coarse_spike_filter(displacement)
##########################################
    
    xmean, xstdev = StatsBase.mean_and_std(displacement)
    suspects = findall(>(xmean + xstdev*3.5), abs.(displacement))
    [displacement[s] = 0.0 for s ∈ suspects]

    return(displacement)    # coarse_spike_filter()

end


# Function for splitting filename and extracting details
function extract_details(infil)
###############################
    
    split_details = split(infil, "\\")
    site_name = split_details[end][1:4]
    date_string = split_details[end][6:15]
    time_string = replace(split_details[end][17:21], 'h' => ':')
    date = DateTime(date_string * " " * time_string, "yyyy-mm-dd HH:MM")
    return(site_name, date)
    
end    # extract_details()


# Calculate spread and direction from Fourier coefficients
function calc_spread_direction(a1, b1)
######################################
    
    θ₀ = atan.(b1,a1)
    m1 = (a1.^2 .+ b1.^2).^0.5
##    m2 = a2.*cos.(2*θ₀) .+ b2.*sin.(2*θ₀)
##    n2 = -a2.*sin.(2*θ₀) .+ b2.*cos.(2*θ₀)

    # Calculate the spread
    σc = (2 .* (1 .- m1)).^0.5;

    direction = mod.(rad2deg.(atan.(b1,a1)),360)

    return(σc, direction)

end    # calc_spread_direction()


#######################################################################################################
#######################################################################################################
#######################################################################################################

# Define the path to the directory you want to search in
directory_path = pick_folder()

# Use glob to find all .HXV files in the directory and subdirectories
hxv_files = glob("**/*.hxv", directory_path)

# build df containing displacements and Fourier coefficient for selected day
global displacement_df = DataFrame(Date = [], Heave = [], North = [], West = [], fhh = [], Chh = [], 
    a1 = [], b1 = [], a2 = [], b2 = [], f2 = [], Pden2 = [], Spread = [], Direction = [])

println("Reading all .HXV files for selected month (this takes a while!):")
flush(stdout)

@time begin
fl_num = 1
    
for fl ∈ hxv_files
    
    infil = fl
    site_name, date = extract_details(infil)
    
    if (mod(fl_num,10) == 0)
        print(string(fl_num))
    else
        print(".")
    end
        
    heave, north, west = get_HNW(infil)
            
########################################################################### 
    heave = coarse_spike_filter(heave)
    north = coarse_spike_filter(north)
    west = coarse_spike_filter(west)
###########################################################################
            
    fhh, Chh, a1, b1, a2, b2 = get_Fourier_coefficients(heave, north, west)
    
    σc, direction = calc_spread_direction(a1, b1)
    
    # Present spectra using Welch's method to better define bimodal events
    ps_w = welch_pgram(heave, 256, 128; onesided=true, nfft=256, fs=sample_frequency, window=hanning);
    f2 = freq(ps_w);
    Pden2 = power(ps_w)
    
    push!(displacement_df, (date, heave, north, west, fhh, Chh, a1, b1, a2, b2, f2, Pden2, σc, direction))
    fl_num += 1
            
end
end
println("Done!")

### Use Serialize to read .bin file to df

In [ ]:
using AbstractFFTs
using CodecZlib: GzipDecompressorStream
using Serialization: deserialize
using NativeFileDialog: pick_file
using DataFrames: DataFrame
using Dates: DateTime

##include("../Split_Spectra/Split_Spectra_Tools.jl")
##include("C://Users//PC1//Julia_programs//Split_Spectra//Split_Spectra_Tools.jl")

# Widen screen for better viewing
display(HTML("<style>:root { --jp-notebook-max-width: 80% !important; }</style>"))

function read_bin_file(io)
###########################
    
    #gz = GzipDecompressorStream(io)                # Create a Gzip decompressor stream
    deserialized_RDT_df = deserialize(io) # Deserialize the DataFrame from the decompressed stream
    close(io)                                      # Close the decompressor stream
    
    return(deserialized_RDT_df)
    
end    # read_gzip_file()


#######################################################################################################
#######################################################################################################
#######################################################################################################


# Select the binary file
##infil = pick_file_from_dirs()
path = @__DIR__
binary_infil = pick_file(path * "\\Data")

println("Selected ", binary_infil)
flush(stdout)

# declare the Heave, North, and West values from the Hex data
deserialized_RDT_df = DataFrame(
    Date = DateTime[], 
    Heave = Vector{Float64}[], 
    North = Vector{Float64}[], 
    West = Vector{Float64}[], 
    GPS_flag = Vector{Int16}[], 
    GPS_errors = Int64[], 
    f2 = Vector{Float64}[], 
    Pden2 = Vector{Float64}[], 
)

@time begin
    
    # Deserialize the DataFrame from the file
    displacement_df = open(read_bin_file, binary_infil, "r")
    
end;

### Use Serialize to write df to .bin file

In [ ]:
using Serialization: serialize
using DataFrames
using NativeFileDialog
using Dates, DataFrames, Distributions, DSP
using LaTeXStrings
using Printf

# Serialize the DataFrame to a file
##outfil = ".\\Data\\" * split(directory_path,"\\")[end-1]*"_"*Dates.format(Date(Date(displacement_df.Date[1])), "yyyy-mm-dd")*
##    "_to_"*Dates.format(Date(Date(displacement_df.Date[end])), "yyyy-mm-dd")*".bin"

outfil = "C:\\Users\\PC1\\Julia_programs\\Datawell\\RDT_vector\\Data\\Noise_floor.bin"

println("Writing binary-formatted data to ",outfil)
flush(stdout)

@time begin

    open(outfil, "w") do io
        serialize(io, deserialized_RDT_df)
    end

end

### Use this to read Noise_Floor

In [ ]:
function read_gzip_file(io)
###########################
    
##    gz = GzipDecompressorStream(io)                # Create a Gzip decompressor stream
    deserialized_RDT_df = deserialize(io) # Deserialize the DataFrame from the decompressed stream
    close(io)                                      # Close the decompressor stream
    
    return(deserialized_RDT_df)
    
end    # read_gzip_file()


path = @__DIR__
noise_floor_infil = path * "\\Data\\Noise_floor.bin"
println("Reading Noise Floor data from ",noise_floor_infil)
flush(stdout)

@time begin
    
    # Deserialize the DataFrame from the file
    noise_floors_df = open(read_gzip_file, noise_floor_infil, "r");
    
end;

# Verify the contents
##println(noise_floors_df)


### Plot spectrogram for month

In [ ]:
using Plots

gr()

display(HTML("<style>:root { --jp-notebook-max-width: 80% !important; }</style>"))

dates = displacement_df.Date
freqs = displacement_df.f2[1]
spectra = hcat(displacement_df.Pden2...)
max_spec = maximum(spectra)

start_date =  first(dates)
last_date = last(dates)

# display plots to screen
tm_tick = range(start_date,last_date,step=Day(1))
ticks = Dates.format.(tm_tick,"dd")

#p1 = heatmap(first(displacement_df.Date) + Microsecond.(ceil.((spec.time) * 1000000)), spec.freq, power_spec, lw=0.25, c=cgrad(:Spectral, rev=true), clims=(0.0,max_spec), levels=10, fill=true)
p1 = contourf(dates, freqs, spectra, lw=0.25, c=cgrad(:Spectral, rev=true), clims=(0,max_spec), levels=10, fill=true)
#p1 = Plots.plot!(displacement_df.Date,noise_floors)
# draw grid lines on plot
for i in 0:0.1:0.6
    hline!(p1, [i], lw=0.5, c=:white, label="")
end

for i in start_date:Day(1):last_date
    vline!(p1, [i], lw=0.5, c=:white, label="")
end

#title = Dates.format(start_date, "yyyy-mm-dd") * " to " * Dates.format(last_date, "yyyy-mm-dd")
##title = split(split(hxv_files[1],"\\")[end],"_")[1]*"_"*monthname(displacement_df.Date[1])*"_"*string(year(displacement_df.Date[1]))
title=""
plot_file = ".\\Plots\\"*title*"_monthly_spectrogram.png"

p1_plot = plot(p1, xlabel="Date", xlim=(start_date,last_date), xticks=(tm_tick,ticks), xtickfontsize=7,
        ylabel="Frequency (Hz)", ylim=(0,0.4), ytickfontsize=8, 
        title=title, framestyle = :box,
        leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1800,800), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, colorbar=true)

try
                                                
    savefig(plot_file)
    println("\nPlot file saved as "*plot_file)

catch

    "Alert: Plot not saved!"

end

display(p1_plot)

### Select start day of 5-day spectrogram

In [ ]:
using CSV
using NativeFileDialog
using Glob
using Dates, DataFrames, Distributions, DSP
using LaTeXStrings
using Printf
using Statistics #, StatsPlotss
using StatsBase

dates = displacement_df.Date

# Initialize an empty array to store daily intervals
dates_array = Date[]

# Iterate through dates and extract dates at daily intervals
for i in 1:length(dates)
    if i == 1 || day(dates[i]) != day(dates[i-1])
        push!(dates_array, dates[i])
    end
end

using Tk

dates_array = Dates.format.(dates_array, "yyyy-mm-dd")

w = Toplevel("Select Start Date", 235, 650)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)
lb = Treeview(f1, dates_array)

scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Tk.Button(f, "Ok")
pack(b)

println("Select a time from the menu!")
flush(stdout)

bind(b, "command") do path
                    
    file_choice = get_value(lb);

    global start_date = DateTime(file_choice[1])

end

### Do 5-day spectrogram

### Calculate Noise Floor as median + 3 * MAD

In [ ]:
using DataFrames, Statistics


# Function to calculate noise floor
function calculate_noise_floor(df)
##################################
    
    spectra_data = df.Pden2
    noise_levels = []
    
    for spectra in spectra_data
        # Calculate median and median absolute deviation (MAD)
        median_value = median(spectra)
        mad_value = median(abs.(spectra .- median_value))
        
        # Determine noise floor as median + 3 * MAD (adjust threshold as needed)
        noise_floor = median_value + 3 * mad_value
        push!(noise_levels, noise_floor)
    end
    
    return noise_levels
    
end    # calculate_noise_floor()


#######################################################################################################
#######################################################################################################
#######################################################################################################


# Calculate noise floor for Spectra column
noise_floors = calculate_noise_floor(noise_floors_df);
##println("Noise Floors: ", noise_floors)

println("Done!")

### Calculate mean value of Noise Floor spectra

In [ ]:
# Extract all spectral arrays from the DataFrame
spectral_values = displacement_df.Pden2

# Convert the list of arrays into a matrix where each row is a spectrum
spectral_matrix = hcat(spectral_values...)'

# Calculate the mean spectra (mean of each column)
mean_spectra = mean(spectral_matrix, dims=1)

# Convert the result to a vector (if needed)
mean_spectra_vector = vec(mean_spectra);

##println("Mean Spectra: ", mean_spectra_vector)
println("Done!")

### Calculate Noise Floor Median and MAD

In [ ]:
function mad(arr)
#################
    
    med = median(arr)
    mad_value = median(abs.(arr .- med))
    return mad_value
    
end


function mad_matrix(matrix; dims=1)
###################################
    
    if dims == 1
        return [mad(matrix[:, i]) for i in 1:size(matrix, 2)]
    elseif dims == 2
        return [mad(matrix[i, :]) for i in 1:size(matrix, 1)]
    else
        error("Unsupported dimension: $dims")
    end
    
end


#######################################################################################################
#######################################################################################################
#######################################################################################################


# Extract all spectral arrays from the DataFrame
spectral_values = noise_floors_df.Pden2

# Convert the list of arrays into a matrix where each row is a spectrum
spectral_matrix = hcat(spectral_values...)'

# Calculate the median spectra (median of each column)
median_spectra = median(spectral_matrix, dims=1)

# Calculate the MAD (median absolute deviation) for each frequency
mad_spectra = mad_matrix(spectral_matrix, dims=1)

# Convert the results to vectors
median_spectra_vector = vec(median_spectra)
mad_spectra_vector = vec(mad_spectra);
##println("Median Spectra: ", median_spectra_vector)
##println("MAD Spectra: ", mad_spectra_vector)
println("Done!")

### Remove outliers using MAD

In [ ]:
function remove_outliers_using_mad(matrix, threshold=3.0)
#########################################################
    
    median_vals = median(matrix, dims=1)
    mad_vals = mad_matrix(matrix, dims=1)
    lower_bound = median_vals .- threshold .* mad_vals
    upper_bound = median_vals .+ threshold .* mad_vals
    filtered_matrix = copy(matrix)
    
    for i in 1:size(matrix, 2)  # iterate over columns
        for j in 1:size(matrix, 1)  # iterate over rows
            if matrix[j, i] < lower_bound[i] || matrix[j, i] > upper_bound[i]
                filtered_matrix[j, i] = median_vals[i]
            end
        end
    end
    
    return filtered_matrix
    
end


#######################################################################################################
#######################################################################################################
#######################################################################################################


filtered_spectral_matrix = remove_outliers_using_mad(spectral_matrix)
mean_spectra_filtered = mean(filtered_spectral_matrix, dims=1)
mean_spectra_vector_filtered = vec(mean_spectra_filtered);

##println("Filtered Mean Spectra: ", mean_spectra_vector_filtered)
println("Done!")

### Do log plot of spectra for range of dates and show Noise Floor based on Hay Point buoy data 

### Do Log plot of 10 highest spectra over selected dates

In [ ]:
using Plots: Plots, alpha, plot, plot!, savefig, xlims, xlabel!, ylims, ylabel!
using Dates: Dates

# Function to find the peak spectral value within a specified frequency range
function find_peak_in_range(df, freq_min, freq_max)
###################################################
    
    peak_value = -Inf
    peak_row_index = -1

    for (i, row) in enumerate(eachrow(df))
        frequencies = row.f2
        spectral_values = row.Pden2

        # Find indices within the specified frequency range
        indices_in_range = findall(freq_min .<= frequencies .<= freq_max)
        
        if !isempty(indices_in_range)
            # Get spectral values within the specified range
            spectral_values_in_range = spectral_values[indices_in_range]

            # Find the peak spectral value in the range
            max_value_in_range = maximum(spectral_values_in_range)

            # Update the peak value and row index if a new peak is found
            if max_value_in_range > peak_value
                peak_value = max_value_in_range
                peak_row_index = i
            end
        end
    end

    return peak_row_index, peak_value
    
end    # find_peak_in_range()


# Function to find the top N peak spectral values within a specified frequency range
function find_top_peaks_in_range(df, freq_min, freq_max, top_n)
###############################################################

    peak_indices = []

    for row in eachrow(df)
        
        spectra = row.Pden2
        frequencies = row.f2
        
        # Filter indices where frequency is between defined minimum and maximum values
        global valid_indices = findall(x -> freq_min <= x <= freq_max, frequencies)
    
       # Get the corresponding spectra values
        max_index = argmax(spectra[valid_indices])#    print(frequencies[max_index]," ")
    
        # Push the peak value to the peak_indices array
        push!(peak_indices, spectra[valid_indices][max_index])
      
    end

    df.peak_values = peak_indices

    return(sort(filtered_df, :peak_values, rev=true)[1:top_n, :])

end    # find_top_peaks_in_range()


# Function to convert frequency in Hertz to Period in Seconds
function convert_frequency_to_period(frequencies)
#################################################
        
    return(1.0 ./ frequencies)
        
end    # convert_frequency_to_period()


# Check if a file name contains the word “_corrected”    
function file_contains_word(file_path::String, word::String)::Bool
##################################################################
    
    return occursin(word, basename(file_path))
    
end    # file_contains_word()


#######################################################################################################
#######################################################################################################
#######################################################################################################

##displacement_df = deserialized_RDT_df
    
# Define the date range
##start_date = DateTime("2009-05-24")
##end_date = DateTime("2009-05-26")
start_date = displacement_df.Date[1]
end_date = displacement_df.Date[end]

p1 = plot()
    
@time begin
    
    # Filter the DataFrame
    filtered_df = displacement_df[(displacement_df.Date .>= start_date) .& (displacement_df.Date .<= end_date), :];
    
    # Define the frequency range
    freq_min = 0.0
    freq_max = 0.05
    
    # Find the row with the peak spectral value in the specified frequency range
    ##peak_row_index, peak_value = find_peak_in_range(filtered_df, freq_min, freq_max)
    
    # Number of top peaks to find
    top_n = 10
    
    # Find the top N rows with the peak spectral values in the specified frequency range
    top_peaks = find_top_peaks_in_range(filtered_df, freq_min, freq_max, top_n)
    
    #==
    # Display the result
    if peak_row_index != -1
        println("Row with peak spectral value within the range $freq_min to $freq_max Hz:")
        println(filtered_df[peak_row_index, :])
        println("Peak spectral value: $peak_value")
    else
        println("No peak found within the range $freq_min to $freq_max Hz.")
    end
    ==#
    
    # Plotting each row
    for row in eachrow(filtered_df)
        frequencies_hz = row.f2[2:11]
        spectral_values = row.Pden2[2:11]
        periods_sec = convert_frequency_to_period(frequencies_hz)
        
        # Create plot for the current row
        p1 = plot!(periods_sec, spectral_values, lw=:1, lc=:lightgrey, alpha=0.5, label="")
    end
    
    
    for row in eachrow(top_peaks)
        frequencies_hz = row.f2
        spectral_values = row.Pden2
        periods_sec = convert_frequency_to_period(frequencies_hz)
        peak_time_string = Dates.format(row.Date, "yyyy-mm-dd HH:MM")
        p1 = plot!(periods_sec, spectral_values, label=peak_time_string, lw=:3, alpha=0.5,)
    end
    
    # Plot the mean of the Noise Floor values
    periods_sec = convert_frequency_to_period(filtered_df.f2[1])
    p1 = plot!(periods_sec,  mean_spectra_vector, lw=:2, lc=:grey, ls=:dash, label="Noise Floor Mean")
    
    Dates.format(start_date, "yyyy-mm-dd HH:MM:SS")
    title = Dates.format(start_date, "yyyy-mm-dd")*" to "*Dates.format(end_date, "yyyy-mm-dd")*" AEST"
    
    p1_plot = plot(p1, xtickfontsize=7, title=title,
        ytickfontsize=8, yaxis=:log, xlims=(20,200), xminorticks=5, legend=:bottomright,
        fg_legend=:transparent, bg_legend=:transparent, plot_padding=1Plots.mm,
        framestyle = :box, xlabel="Period (Seconds)", ylabel="Spectral Density (m²/Hertz)",
        leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1800,800), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)
    
    try
        # check whether binary file have been corrected for GPS errors or not
        if file_contains_word(binary_infil, "_corrected")
            plot_file = replace(replace(title*"_longest_period_wave_corrected.png", "-" => "_"), " " => "_")
        else
            plot_file = replace(replace(title*"_longest_period_wave_uncorrected.png", "-" => "_"), " " => "_")
        end
        
        path = @__DIR__
        savefig(path * "\\Plots\\" * plot_file)
        println("\nPlot file saved as "*plot_file)
        flush(stdout)
    
    catch
    
        "Alert: Plot not saved!"
    
    end
    
    display(p1_plot)

end

In [ ]:
# Define the date range
##start_date = DateTime("2009-05-24")
##end_date = Date("2009-05-26")

start_date = displacement_df.Date[1]
end_date = displacement_df.Date[end]

# Filter the DataFrame
filtered_df = displacement_df[(displacement_df.Date .>= start_date) .& (displacement_df.Date .<= end_date), :]

p1 = Plots.plot()

# Function to convert frequency in Hertz to Period in Seconds
function convert_frequency_to_period(frequencies)
#################################################
    
    return 1.0 ./ frequencies
    
end    # convert_frequency_to_period()


# Plotting each row
for row in eachrow(filtered_df)
    
    frequencies_hz = row.f2[2:11]
    spectral_values = row.Pden2[2:11]
    periods_sec = convert_frequency_to_period(frequencies_hz)
    
    # Create plot for the current row
    p1 = Plots.plot!(periods_sec, spectral_values, label="", lw=:2, lc=:lightgrey, alpha=0.5,)
    
end

# Plotting each row
for row in eachrow(filtered_df)
    frequencies_hz = row.f2[2:11]
    spectral_values = row.Pden2[2:11]
    periods_sec = convert_frequency_to_period(frequencies_hz)
    
    # Create plot for the current row
    p1 = Plots.plot!(periods_sec, spectral_values, label="", lw=:1, lc=:lightgrey, alpha=0.5,)
end


for row in eachrow(top_peaks)
    frequencies_hz = row.f2
    spectral_values = row.Pden2
    periods_sec = convert_frequency_to_period(frequencies_hz)
    peak_time_string = Dates.format(row.Date, "yyyy-mm-dd HH:MM")
    p1 = Plots.plot!(periods_sec, spectral_values, label=peak_time_string, lw=:3, alpha=0.5,)
end

# Plot the mean of the Noise Floor values
periods_sec = convert_frequency_to_period(filtered_df.f2[1])
p1 = Plots.plot!(periods_sec,  mean_spectra_vector, lw=:2, lc=:grey, ls=:dash, label="Noise Floor Mean")

Dates.format(start_date, "yyyy-mm-dd HH:MM:SS")
title = Dates.format(start_date, "yyyy-mm-dd")*" to "*Dates.format(end_date, "yyyy-mm-dd")
plot_file = ".\\Plots\\"*title*"_sea_swell_split.png"

p1_plot = plot(p1, xtickfontsize=7, title=title,
    ytickfontsize=8, xlims=(20,200), xminorticks=5, legend=:topright, # ylims=(0, 0.1),
    fg_legend=:transparent, bg_legend=:transparent, plot_padding=1Plots.mm,
    framestyle = :box, xlabel="Period (Seconds)", ylabel="Spectral Density (m²/Hertz)",
    leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1800,800), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)


plot_file = ".\\longest_period_wave.png"
try
                                                
    savefig(plot_file)
    println("\nPlot file saved as "*plot_file)

catch

    "Alert: Plot not saved!"

end

display(p1_plot)

In [ ]:
# Define the frequency range
freq_min = 0.0
freq_max = 0.05

# Find the row with the peak spectral value in the specified frequency range
peak_row_index, peak_value = find_peak_in_range(filtered_df, freq_min, freq_max)

# Display the result
if peak_row_index != -1
    
    println("Row with peak spectral value within the range $freq_min to $freq_max Hz:")
    println(filtered_df[peak_row_index, :])
    println("Peak spectral value: $peak_value")
    
else
    
    println("No peak found within the range $freq_min to $freq_max Hz.")
    
end


In [ ]:
using DataFrames
using Statistics
using FFTW

# Define a function to calculate MAD for a 1D array
function mad(arr)
#################
    
    med = median(arr)
    mad_value = median(abs.(arr .- med))
    return mad_value
    
end    # mad()

# Define a function to calculate MAD for a matrix
function mad_matrix(matrix; dims=1)
###################################
    
    if dims == 1
        return [mad(matrix[:, i]) for i in 1:size(matrix, 2)]
    elseif dims == 2
        return [mad(matrix[i, :]) for i in 1:size(matrix, 1)]
    else
        error("Unsupported dimension: $dims")
    end
    
end    # mad_matrix()


# Define a function to remove outliers using MAD
function remove_outliers_using_mad(matrix, threshold=3.0)
#########################################################
    
    median_vals = median(matrix, dims=1)
    mad_vals = mad_matrix(matrix, dims=1)
    lower_bound = median_vals .- threshold .* mad_vals
    upper_bound = median_vals .+ threshold .* mad_vals
    filtered_matrix = copy(matrix)
    
    for i in 1:size(matrix, 2)  # iterate over columns
        for j in 1:size(matrix, 1)  # iterate over rows
            if matrix[j, i] < lower_bound[i] || matrix[j, i] > upper_bound[i]
                filtered_matrix[j, i] = median_vals[i]
            end
        end
    end
    
    return filtered_matrix
    
end    # remove_outliers_using_mad()


#######################################################################################################
#######################################################################################################
#######################################################################################################

    
# Extract all spectral arrays from the DataFrame
spectral_values = displacement_df.Pden2

# Convert the list of arrays into a matrix where each row is a spectrum
spectral_matrix = hcat(spectral_values...)'

# Calculate the mean spectra (mean of each column) directly
mean_spectra = mean(spectral_matrix, dims=1)
mean_spectra_vector = vec(mean_spectra)

# Calculate the median spectra (median of each column)
median_spectra = median(spectral_matrix, dims=1)

# Calculate the MAD (median absolute deviation) for each frequency
mad_spectra = mad_matrix(spectral_matrix, dims=1)

# Convert the results to vectors
median_spectra_vector = vec(median_spectra)
mad_spectra_vector = vec(mad_spectra)

println("Mean Spectra: ", mean_spectra_vector)
println("Median Spectra: ", median_spectra_vector)
println("MAD Spectra: ", mad_spectra_vector)

# Remove outliers from the spectral matrix
filtered_spectral_matrix = remove_outliers_using_mad(spectral_matrix)

# Calculate the mean spectra after outlier removal
mean_spectra_filtered = mean(filtered_spectral_matrix, dims=1)
mean_spectra_vector_filtered = vec(mean_spectra_filtered)

##println("Filtered Mean Spectra: ", mean_spectra_vector_filtered)

# Compute the difference spectra
difference_spectra = mean_spectra_vector - mean_spectra_vector_filtered

##println("Difference Spectra: ", difference_spectra)

# Perform the inverse Fourier transform to get the time series of the outliers
time_series_outliers = real(ifft(difference_spectra))

Plots.plot(time_series_outliers)


### atan2() function

In [ ]:
# Calculate atan2 in Radians
############################
function atan2(y::Real, x::Real)
    
    if x > 0
        
        return atan(y / x)
        
    elseif x < 0
        
        if y >= 0
            
            return atan(y / x) + π
        else
            
            return atan(y / x) - π
            
        end
        
    else  # x == 0
        
        if y > 0
            
            return π / 2
            
        elseif y < 0
            
            return -π / 2
            
        else  # y == 0
            
            return NaN  # Error condition

        end

    end

end    # atan2()


for i in 1:1
    println((360 .+ rad2deg.(atan2.(displacement_df.b1[i],displacement_df.a1[i]))) .% 360)
end

In [ ]:
using Plots

Plots.plot((360 .+ rad2deg.(atan2.(displacement_df.b1[1],displacement_df.a1[1]))) .% 360)
Plots.plot!(displacement_df.Direction[1])